In [1]:
from gurobipy import *  # import the optimize solver Gurobi

In [2]:
number_of_year = 4 # Set the index i for number of year(1, 2, 3, 4)
m = Model() # Import and create the model

Using license file C:\Users\chaoz\gurobi_licence\gurobi.lic
Academic license - for non-commercial use only


In [3]:
# Set the input Parameter: 
cost_hire = 40000  # Cost to hire a teacher 
cost_fire = 20000  # Cost to fire a teacher 
salary = 100000  # # Salary of a teacher
demand = [6000, 7000, 5000, 6500]   # Demand for teacher in each year 
initial_teachers = 5000  # Number of teachers available at the beginning of year 1 

In [4]:
# Set the Variable list: Number of teachers to hire in each year nh; Number of teachers to fire in each year nf;
# Set the variable nh and nf to integer number
nh = [] 
for i in range(number_of_year):
    nh.append(m.addVar(vtype=GRB.INTEGER, name='nh{}'.format(i + 1)))

nf = [] 
for i in range(number_of_year):
    nf.append(m.addVar(vtype=GRB.INTEGER, name='nf{}'.format(i + 1)))

# Caculate number of teachers working at the end of each year nw
# nw = []
# for i in range(len(nh)):
#     if i > 0:
#         initial_teachers = nw[i-1]
#     xx = nh[i] + initial_teachers - nf[i]
#     nw.append(xx)
nw = []
for i in range(len(nh)):
    if i <= 0:
        xx = nh[i] + initial_teachers  - nf[i]
    else:
        xx = nh[i] + nw[i - 1] - nf[i]
    nw.append(xx)


In [5]:
# Set the Minimize Obijective: Total Cost
m.setObjective(quicksum([cost_hire*nh[i] for i in range(len(nh))]) + quicksum([cost_fire*nf[i] for i in range(len(nh))]) + quicksum([salary*nw[i] for i in range(len(nh))]),  GRB.MINIMIZE)

In [6]:
# Set Non Negative decision variable
c1 = []
for i in range(len(nh)):
    c1.append(m.addConstr(nh[i] >= 0))

c2 = []
for i in range(len(nf)):
    c2.append(m.addConstr(nf[i] >= 0))
    
# Demand for teachers must be satisfied for each year 
c3 = []
for i in range(len(nh)):
    c3.append(m.addConstr(nw[i] >= demand[i]))

In [7]:
# Run the optimize solver
m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 12 rows, 8 columns and 28 nonzeros
Model fingerprint: 0x2319e6f1
Variable types: 0 continuous, 8 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+04, 4e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 2e+03]
Found heuristic solution: objective 2.690000e+09
Presolve removed 12 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 2.63e+09 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.630000000000e+09, best bound 2.630000000000e+09, gap 0.0000%


In [8]:
# Get the Optimal Solution for X
m.printAttr('X')


    Variable            X 
-------------------------
         nh1         1000 
         nh2         1000 
         nh4         1500 
         nf3         2000 


In [9]:
# Get the Optimal Objective Value
m.ObjVal

2630000000.0